In [74]:
import math
import random
import datetime
import seaborn as sns
import copy
from datetime import datetime
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
'''
The enrollment data file was transformed to contain one daw per row and named days_sliced. It is taken as input.
time of day where student has no class is the output
'''

In [77]:
st = pd.read_excel('days_sliced.xlsx')

In [78]:
        #PREPROCESSING 
st['STUDENTSOURCEKEY'] = st['STUDENTSOURCEKEY'].astype(int) '''<-- converting sourcekey column to integer type'''
st['TIMES'] = st['TIMES'].astype(int) '''<-- time column to integer type'''
st['CREDITSATTEMPTED'] = st['CREDITSATTEMPTED'].astype(int)  '''credits column to integer type'''
'''Selecting desired columns'''
st = st[['STUDENTSOURCEKEY','CREDITSATTEMPTED','CLASSMEETINGPATTERN', 'CLASSSTARTTIME','CLASSENDTIME',  'TIMES']]
st = st.sort_values(by = 'CLASSSTARTTIME').reset_index()

In [79]:
###### CREATING AND POPULATING A DICTIONARY OF CLASS TIMES TAKES LONG TIME
main = defaultdict(list)
for i in st.index:
#     print(i)
#     print(st.loc[i]['CLASSMEETINGPATTERN'].lower())
    start = st.loc[i]['CLASSSTARTTIME']
    end = st.loc[i]['CLASSENDTIME']
    main[(st.loc[i]['STUDENTSOURCEKEY'],st.loc[i]['CLASSMEETINGPATTERN'].lower())].append((start,end))

In [81]:
#########  CREATING  DICTIONARY OF FREE TIME FOR EACH DAY
'''
Iterate over the day and select first class time starting  from 9 AM.
make the end of first class as start of first interval and start of next class as end of the current interval. 
Repeat timm end time (6 PM).
'''
#########JOINING THE TIMES TOGETHER AS TUPLES
amputed=defaultdict(list)
for key,times in main.items():
    res=[]
    prev=time.strptime('9:00AM', '%I:%M%p') '''<-- set start time of day here'''
    for t in times:
        s1 = t[0].strftime("%I:%M%p")
        start = time.strptime(s1,'%I:%M%p')
        e1 = t[1].strftime("%I:%M%p")
        end = time.strptime(e1,'%I:%M%p')
        if prev>=time.strptime('6:00PM', '%I:%M%p'): '''<-- set end time of the day here'''
            break
        if prev<start:
            res.append((str(prev.tm_hour)+':'+str(prev.tm_min),str(start.tm_hour)+':'+str(start.tm_min)))
        if end>prev:
            prev=end
    if prev<time.strptime('6:00PM', '%I:%M%p'):     
        res.append((str(prev.tm_hour)+':'+str(prev.tm_min),'18:00'))
    amputed[key]+=res
# amputed


In [82]:
'''The dictionary with times of free times each day is converted to data frame'''
data = pd.DataFrame.from_dict(amputed,orient='index').reset_index()
data[['ID', 'day']] = pd.DataFrame(data['index'].tolist())
data.drop(columns='index',inplace=True)
data = data[['ID', 'day',0, 1, 2, 3, 4, 5, 6]]
data = data.fillna(0)
'''Since the co-class times are given as start and end time tuple pairs, 
    duration of each intervalis calculated seperately here.'''
def func(x):
    if x==0:
        return 0
    else:
#         s = datetime.strp(x[0],'%H:%M')
        return (datetime.strptime(x[1],'%H:%M') -datetime.strptime(x[0],'%H:%M')).total_seconds()/60
for i in range(l_):
    temp = 'dur_' + str(i)
    data[temp] = data[i].apply(lambda x: func(x))
'''Selecting only the required columns '''
fin= data[['ID', 'day',0, 'dur_0',1,'dur_1',2,'dur_2',3,'dur_3', 4,'dur_4',5,'dur_5',6,'dur_6',]]
'''Writing the dataframe to excel'''
fin.to_excel('no_class.xlsx')